## 정수 인코딩

### 1. dictionary 사용하기
- 위키피디아에 문자 인코딩 검색한 개요를 테스트 문장으로 사용

In [2]:
# konlpy를 사용하려면 자바를 설치하고 본인 경로에 맞게 아래와 같이 설정을 해줘야한다.
# echo 'export JAVA_HOME=$(/usr/libexec/java_home)' >> .zshrc / 또는 .bashrc
from konlpy.tag import Okt
okt=Okt()


In [3]:
text=[
'문자 인코딩, 줄여서 인코딩은 사용자가 입력한 문자나 기호들을 컴퓨터가 이용할 수 있는 신호로 만드는 것을 말한다.',
'넓은 의미의 컴퓨터는 이러한 신호를 입력받고 처리하는 기계를 뜻하며, 신호 처리 시스템을 통해 이렇게 처리된 정보를 사용자가 이해할 수 있게 된다.', 
'이 신호를 입력하는 인코딩과 문자를 해독하는 디코딩하기 위해선 미리 정해진 기준을 바탕으로 입력과 해독이 처리되어야 하는데, 이를 문자열 세트 또는 문자셋이라고 한다.', 
'초기 보급형 컴퓨터의 문자열 세트는 아스키나 EBCDIC이 표준이었으나, 이러한 문자열 세트는 세계 곳곳에 인터넷이 보급되며 표현해야 할 문자가 증가하면서 이러한 문자셋들을 표준화하기 위해 많은 대체 방식이 개발되었다.',
'표준 문자셋을 개발하는 것에 대한 논의가 이어졌고, 후에는 유니코드가 등장하게 되었다.'
]

In [12]:
from collections import Counter
# 불용어
stop_words=['것','수','이','줄','후']
results=[]
# 정제된 코퍼스
result=[]
for line in text:
    tmp=[]
    # 명사 추출
    word_list=okt.nouns(line)
    # 각 단어 불용어 처리 후 저장
    for word in word_list:
        if word not in stop_words:
            tmp.append(word)
            result.append(word)
    results.append(tmp)    
# BoW 화
cnt=Counter(result)

# 빈도수 기준 내림차순 정렬 / Counter 사용시 이미 내림차순 정렬됨.
vocab_sorted=sorted(cnt.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('문자', 4), ('입력', 4), ('신호', 4), ('처리', 4), ('인코딩', 3), ('컴퓨터', 3), ('문자열', 3), ('세트', 3), ('문자셋', 3), ('사용자', 2), ('해독', 2), ('위해', 2), ('보급', 2), ('표준', 2), ('개발', 2), ('기호', 1), ('이용', 1), ('말', 1), ('의미', 1), ('기계', 1), ('뜻', 1), ('시스템', 1), ('통해', 1), ('정보', 1), ('이해', 1), ('디코딩', 1), ('선', 1), ('미리', 1), ('정해진', 1), ('기준', 1), ('바탕', 1), ('초기', 1), ('아스키', 1), ('세계', 1), ('곳곳', 1), ('인터넷', 1), ('표현', 1), ('증가', 1), ('표준화', 1), ('대체', 1), ('방식', 1), ('대한', 1), ('논의', 1), ('유니코드', 1), ('등장', 1)]


In [13]:
word_to_index={}
index_to_word={}

# 2개 이상만 사전에 포함시키기.
for idx,(word,freq) in enumerate(vocab_sorted):
    if freq<2:
        continue
    word_to_index[word]=idx+1
    index_to_word[idx+1]=word
    
# 없는 단어 인덱스 추가
word_to_index['OOV'] = len(word_to_index) + 1
index_to_word[len(index_to_word)+1] = 'OOV'

word_to_index, index_to_word

({'문자': 1,
  '입력': 2,
  '신호': 3,
  '처리': 4,
  '인코딩': 5,
  '컴퓨터': 6,
  '문자열': 7,
  '세트': 8,
  '문자셋': 9,
  '사용자': 10,
  '해독': 11,
  '위해': 12,
  '보급': 13,
  '표준': 14,
  '개발': 15,
  'OOV': 16},
 {1: '문자',
  2: '입력',
  3: '신호',
  4: '처리',
  5: '인코딩',
  6: '컴퓨터',
  7: '문자열',
  8: '세트',
  9: '문자셋',
  10: '사용자',
  11: '해독',
  12: '위해',
  13: '보급',
  14: '표준',
  15: '개발',
  16: 'OOV'})

In [15]:
# 정수 인코딩
encoded = []
for s in results:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5, 5, 10, 2, 1, 16, 6, 16, 3, 16], [16, 6, 3, 2, 4, 16, 16, 3, 4, 16, 16, 4, 16, 10, 16], [3, 2, 5, 1, 11, 16, 12, 16, 16, 16, 16, 16, 2, 11, 4, 7, 8, 9], [16, 13, 6, 7, 8, 16, 14, 7, 8, 16, 16, 16, 13, 16, 1, 16, 9, 16, 12, 16, 16, 15], [14, 9, 15, 16, 16, 16, 16]]


### Keras 텍스트 전처리

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

# 위의 과정 진행. 빈도수 높은 순으로 인덱스 부여
tokenizer.fit_on_texts(results)
print(tokenizer.word_index)

{'문자': 1, '입력': 2, '신호': 3, '처리': 4, '인코딩': 5, '컴퓨터': 6, '문자열': 7, '세트': 8, '문자셋': 9, '사용자': 10, '해독': 11, '위해': 12, '보급': 13, '표준': 14, '개발': 15, '기호': 16, '이용': 17, '말': 18, '의미': 19, '기계': 20, '뜻': 21, '시스템': 22, '통해': 23, '정보': 24, '이해': 25, '디코딩': 26, '선': 27, '미리': 28, '정해진': 29, '기준': 30, '바탕': 31, '초기': 32, '아스키': 33, '세계': 34, '곳곳': 35, '인터넷': 36, '표현': 37, '증가': 38, '표준화': 39, '대체': 40, '방식': 41, '대한': 42, '논의': 43, '유니코드': 44, '등장': 45}


In [17]:
# 등록된 사전에서 해당 인덱스 나열
print(tokenizer.texts_to_sequences(results))

[[1, 5, 5, 10, 2, 1, 16, 6, 17, 3, 18], [19, 6, 3, 2, 4, 20, 21, 3, 4, 22, 23, 4, 24, 10, 25], [3, 2, 5, 1, 11, 26, 12, 27, 28, 29, 30, 31, 2, 11, 4, 7, 8, 9], [32, 13, 6, 7, 8, 33, 14, 7, 8, 34, 35, 36, 13, 37, 1, 38, 9, 39, 12, 40, 41, 15], [14, 9, 15, 42, 43, 44, 45]]


In [29]:
# 케라스 토크나이저는 OOV에 대해서는 아예 단어를 제거한다. OOV를 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용
vocab_size = 5
tok = Tokenizer(num_words=vocab_size+2, oov_token = 'OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tok.fit_on_texts(results)
# vocab_size를 작게해도 word_index, word_counts, word_docs 와 같은
# fit_on_texts 출력 에는 영향을 주지 않습니다 texts_to_matrix에 영향을 미침
print(tok.word_index)
tok.texts_to_matrix(results, mode='count')

{'OOV': 1, '문자': 2, '입력': 3, '신호': 4, '처리': 5, '인코딩': 6, '컴퓨터': 7, '문자열': 8, '세트': 9, '문자셋': 10, '사용자': 11, '해독': 12, '위해': 13, '보급': 14, '표준': 15, '개발': 16, '기호': 17, '이용': 18, '말': 19, '의미': 20, '기계': 21, '뜻': 22, '시스템': 23, '통해': 24, '정보': 25, '이해': 26, '디코딩': 27, '선': 28, '미리': 29, '정해진': 30, '기준': 31, '바탕': 32, '초기': 33, '아스키': 34, '세계': 35, '곳곳': 36, '인터넷': 37, '표현': 38, '증가': 39, '표준화': 40, '대체': 41, '방식': 42, '대한': 43, '논의': 44, '유니코드': 45, '등장': 46}


array([[ 0.,  5.,  2.,  1.,  1.,  0.,  2.],
       [ 0.,  9.,  0.,  1.,  2.,  3.,  0.],
       [ 0., 12.,  1.,  2.,  1.,  1.,  1.],
       [ 0., 21.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  7.,  0.,  0.,  0.,  0.,  0.]])